# Procesamiento de imágenes y texto con VLMs  

Este notebook demuestra cómo utilizar el modelo `HuggingFaceTB/SmolVLM-Instruct` cuantizado a 4 bits para diversas tareas multimodales, como:  

- **Visual Question Answering (VQA)**: Responder preguntas basadas en el contenido de una imagen.  
- **Reconocimiento de Texto (OCR)**: Extraer e interpretar texto en imágenes.  
- **Comprensión de Video**: Describir videos mediante el análisis secuencial de fotogramas.  

Al estructurar los prompts de manera efectiva, puedes aprovechar el modelo para múltiples aplicaciones, como comprensión de escenas, análisis de documentos y razonamiento visual dinámico.

In [ ]:
# Instalar los requisitos en Google Colab
# !pip install transformers datasets trl huggingface_hub bitsandbytes

# Autenticarse en Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch, PIL
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig
from transformers.image_utils import load_image

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "HuggingFaceTB/SmolVLM-Instruct"
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quantization_config,
).to(device)
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

print(processor.image_processor.size)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
You shouldn't move a model that is dispatched using accelerate hooks.
Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


{'longest_edge': 1536}


## Procesamiento de Imágenes

Comencemos generando descripciones y respondiendo preguntas sobre una imagen. También exploraremos el procesamiento de múltiples imágenes.

### 1. Generación de Descripción para una Imagen Única

In [3]:
from IPython.display import Image, display

image_url1 = "https://cdn.pixabay.com/photo/2024/11/20/09/14/christmas-9210799_1280.jpg"
display(Image(url=image_url1))

image_url2 = "https://cdn.pixabay.com/photo/2024/11/23/08/18/christmas-9218404_1280.jpg"
display(Image(url=image_url2))

In [ ]:
# Cargar una imagen
image1 = load_image(image_url1)

# Crear mensajes de entrada
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"}
        ]
    },
]

# Preparar las entradas
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(device)

# Generar salidas
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

# Imprimir el texto generado
print(generated_texts)

/home/duydl/Miniconda3/envs/py310/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


['User:<image>Can you describe the image?\nAssistant: The image is a scene of a person walking in a forest. The person is wearing a coat and a cap. The person is holding the hand of another person. The person is walking on a path. The path is covered with dry leaves. The background of the image is a forest with trees.']


### 2. Comparación de Múltiples Imágenes  
El modelo puede procesar y comparar múltiples imágenes. Determinemos la temática en común de dos imágenes.


In [ ]:
# Cargar imágenes
image2 = load_image(image_url2)

# Crear mensajes de entrada
messages = [
    # {
    #     "role": "user",
    #     "content": [
    #         {"type": "image"},
    #         {"type": "image"},
    #         {"type": "text", "text": "Can you describe the two images?"}
    #     ]
    # },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "What event do they both represent?"}
        ]
    },
]

# Preparar las entradas
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(device)

# Generar salidas
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

# Imprimir el texto generado
print(generated_texts)


['User:<image>What event do they both represent?\nAssistant: Christmas.']


### 🔠 Reconocimiento de Texto (OCR)  
El VLM también puede reconocer e interpretar texto en imágenes, lo que lo hace adecuado para tareas como el análisis de documentos.  
Puedes probar con imágenes que contengan texto más denso.

In [ ]:
document_image_url = "https://cdn.pixabay.com/photo/2020/11/30/19/23/christmas-5792015_960_720.png"
display(Image(url=document_image_url))

# Cargar la imagen del documento
document_image = load_image(document_image_url)

# Crear el mensaje de entrada para el análisis
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What is written?"}
        ]
    }
]

# Preparar las entradas
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[document_image], return_tensors="pt")
inputs = inputs.to(device)

# Generar salidas
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

# Imprimir el texto generado
print(generated_texts)

['User:<image>What is written?\nAssistant: MERRY CHRISTMAS AND A HAPPY NEW YEAR']


## Procesamiento de videos  

Los Modelos de Visión-Lenguaje (VLMs) pueden procesar videos de forma indirecta extrayendo fotogramas clave y razonando sobre ellos en orden temporal. Aunque los VLMs no tienen las capacidades de modelado temporal de los modelos de video dedicados, aún pueden:  

- Describir acciones o eventos analizando fotogramas muestreados de forma secuencial.  
- Responder preguntas sobre videos basándose en fotogramas representativos.  
- Resumir el contenido de un video combinando descripciones textuales de múltiples fotogramas.  

Probemos con un ejemplo:  

<video width="640" height="360" controls>
  <source src="https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [ ]:
# !pip install opencv-python

In [ ]:
from IPython.display import Video
import cv2
import numpy as np

def extract_frames(video_path, max_frames=50, target_size=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, max_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if target_size:
                frames.append(resize_and_crop(frame, target_size))
            else:
                frames.append(frame)
    cap.release()
    return frames

def resize_and_crop(image, target_size):
    width, height = image.size
    scale = target_size / min(width, height)
    image = image.resize((int(width * scale), int(height * scale)), PIL.Image.Resampling.LANCZOS)
    left = (image.width - target_size) // 2
    top = (image.height - target_size) // 2
    return image.crop((left, top, left + target_size, top + target_size))

# Link al video
video_link = "https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4"

In [ ]:
question = "Describe what the woman is doing."

def generate_response(model, processor, frames, question):

    image_tokens = [{"type": "image"} for _ in frames]
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": "Following are the frames of a video in temporal order."}, *image_tokens, {"type": "text", "text": question}]
        }
    ]
    inputs = processor(
        text=processor.apply_chat_template(messages, add_generation_prompt=True),
        images=frames,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs, max_new_tokens=100, num_beams=5, temperature=0.7, do_sample=True, use_cache=True
    )
    return processor.decode(outputs[0], skip_special_tokens=True)


# Extraer fotogramas del video
frames = extract_frames(video_link, max_frames=15, target_size=384)

processor.image_processor.size = (384, 384)
processor.image_processor.do_resize = False

# Generar respuesta
response = generate_response(model, processor, frames, question)

# Mostrar el resultado
# print("Question:", question)
print("Response:", response)

Response: User: Following are the frames of a video in temporal order.<image>Describe what the woman is doing.
Assistant: The woman is hanging an ornament on a Christmas tree.


## VLMs en español

Como pudiste notar, todos los ejemplos en este notebook usaron texto en inglés. Esto es porque el modelo `SmolVLM-Instruct` que usamos fue entrenado solamente en inglés. 

## 💐 ¡Has Terminado!  

Este notebook demostró cómo utilizar un Modelo de Visión-Lenguaje (VLM) y cómo estructurar prompts para tareas multimodales. Siguiendo los pasos descritos aquí, puedes experimentar con los VLMs y sus aplicaciones.  

### Próximos Pasos para Explorar:  
- Experimenta con más casos de uso de VLMs.  
- Colabora con un colega revisando sus pull requests (PRs).  
- Contribuye a mejorar este material del curso abriendo un issue o enviando un PR para introducir nuevos casos de uso, ejemplos o conceptos.  

¡Feliz exploración! 🌟  